Here we will be implementing a version of CoCa classifier (the current top performer on ImageNet database) by training it on imagenet imstances of our classes and then asking it to classify our 'strange' images.

In [1]:
# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
!pip install -U -q scikeras
!pip install -U -q pillow==8.4.0
#!pip install -U -q tensorflow
##==2.7.0
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [2]:
#from google.colab import files

#files.upload(); # upload your kaggle.json file

#import json

#!mkdir /root/.kaggle/
#!mv kaggle.json /root/.kaggle/kaggle.json
#!chmod 600 ~/.kaggle/kaggle.json
#!kaggle config set -n path -v{/content}

In [3]:
#be careful, the data is like 160gb +
#!kaggle competitions download -c imagenet-object-localization-challenge

#CATS AND DOGS
!wget --no-check-certificate \
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

#HANDS:
downloaded = drive.CreateFile({'id':'1KcMYcNJgtK1zZvfl_9sTqnyBUTri2aP2'}) 
downloaded.GetContentFile('Hands.zip')

--2022-11-28 13:20:38--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.68.128, 142.250.4.128, 74.125.24.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.68.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M  20.9MB/s    in 3.8s    

2022-11-28 13:20:42 (17.2 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [4]:
import io
import os 
import zipfile

cd_zip = '/tmp/cats_and_dogs_filtered.zip'
zip_ref = zipfile.ZipFile(cd_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()

h_zip = 'Hands.zip'
zip_ref = zipfile.ZipFile(h_zip, 'r')
zip_ref.extractall('/tmp/cats_and_dogs_filtered/train')
zip_ref.close()

#!unzip '/tmp/cats_and_dogs_filtered.zip'
#!unzip Hands.zip -d 'cats_and_dogs_filtered/train'

!ls

Hands.zip  sample_data


In [5]:
#setting up correct directories
import numpy as np
#from image import PIL

#FOR THE MOMENT ALL HANDS AARE IN TRAIN. THIS IS FINE WHATEVER
base_dir = '/tmp/cats_and_dogs_filtered'
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

# Directory with our training cat pictures
train_cats_dir = os.path.join(train_dir, 'cats')

# Directory with our training dog pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')

train_hands_dir = os.path.join(train_dir, 'Hands')


# Directory with our validation cat pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')

# Directory with our validation dog pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')

#NOTE: atm all hands in train dir so no path here

print('total training cat images:', len(os.listdir(train_cats_dir)))
print('total training dog images:', len(os.listdir(train_dogs_dir)))
print('total train hand images:', len(os.listdir(train_hands_dir)))
print('total validation cat images:', len(os.listdir(validation_cats_dir)))
print('total validation dog images:', len(os.listdir(validation_dogs_dir)))


total training cat images: 1000
total training dog images: 1000
total train hand images: 11076
total validation cat images: 500
total validation dog images: 500


In [6]:
#PLOT SOME OF THE IMAGES
#code followed closely from: https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=4PIP1rkmeAYS
#%matplotlib inline

#import matplotlib.pyplot as plt
#import matplotlib.image as mpimg
#import pathlib

# Parameters for our graph; we'll output images in a 4x4 configuration
#nrows = 4
#ncols = 4

# Index for iterating over images
#pic_index = 0
#bBaby = io.BytesIo()
# Set up matplotlib fig, and size it to fit 4x4 pics
#fig = plt.gcf()
#fig.set_size_inches(ncols * 4, nrows * 4)

#pic_index += 8
#next_cat_pix = [os.path.join(train_cats_dir, fname) 
#                for fname in os.listdir(train_cats_dir)[pic_index-8:pic_index]]
#next_dog_pix = [os.path.join(train_dogs_dir, fname) 
#                for fname in os.listdir(train_dogs_dir)[pic_index-8:pic_index]]

#for i, img_path in enumerate(next_cat_pix+next_dog_pix):
  # Set up subplot; subplot indices start at 1
#  sp = plt.subplot(nrows, ncols, i + 1)
#  sp.axis('Off') # Don't show axes (or gridlines)

#  img = mpimg.imread(img_path) #EVEN HERE IT ERRORS. CANNOT USE PIL IMAGES FORMAT?????
#  plt.imshow(img)

#plt.show()


In [7]:

import tensorflow as tf
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import PIL
from PIL import Image


# All images will be rescaled by 1./255 for rgb
cdGen = ImageDataGenerator(rescale=1./255)

#WAY STATED IN TUTORIAL TO GET ALL FILES IN

#batch size=all our images
batch_size = len(train_cats_dir) + len(train_dogs_dir) + len(train_hands_dir)

train_generator = cdGen.flow_from_directory(
    directory=train_dir,
    target_size=(256, 256),
    color_mode="rgb",
    batch_size=batch_size,
    class_mode="categorical"
)

num_samples = train_generator.n
num_classes = train_generator.num_classes
input_shape = train_generator.image_shape

classnames = [k for k,v in train_generator.class_indices.items()]

X, y = train_generator.next()
#X = images
#y = classes

Streaming output truncated to the last 5000 lines.
  [1. 1. 1.]
  [1. 1. 1.]]]
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]]
[[[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 ...

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]]

 [[1. 1. 1.]
  [1. 1. 1.]
  [1. 1. 1.]
  ...
  [1

In [8]:

#clean_directory(train_cats_dir)
#clean_directory(train_dogs_dir)
#clean_directory(train_hands_dir)

#x = np.array([np.array(Image.open(fname)) for fname in filelist])
#cnames = os.listdir(train_cats_dir)
#dnames = os.listdir(train_dogs_dir)
#hnames = os.listdir(train_hands_dir)

#Doesn't matter what I do. PIL images do not work
#cats_train = [os.path.join(train_cats_dir, f) for f in cnames]
#hands_train = [os.path.join(train_hands_dir, f) for f in hnames]
#dogs_train = [os.path.join(train_dogs_dir, f) for f in dnames]

#This crashed runtime lol but works in theory
#hands = np.array([np.array(Image.open(fname)) for fname in hands_train])
#cats = np.array([np.array(Image.open(fname)) for fname in cats_train])
#dogs = np.array([np.array(Image.open(fname)) for fname in dogs_train])


In [9]:
!pip install coca-pytorch
!pip install vit-pytorch

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41 kB 316 kB/s 
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76 kB 5.1 MB/s 


In [10]:
import torch

# import vision transformer

from vit_pytorch import ViT
from vit_pytorch.extractor import Extractor

vit = ViT(
    image_size = 256,
    patch_size = 1,
    num_classes = 3,
    dim = 512,
    depth = 6,
    heads = 16,
    mlp_dim = 512
)

vit = Extractor(vit, return_embeddings_only = True, detach = False)

# extractor will enable it so the vision transformer returns its embeddings

# import CoCa and instantiate it

from coca_pytorch.coca_pytorch import CoCa

#FIXED: must enable GPU hardware acceleration in runtime settings or this will not work
coca = CoCa(
    dim = 256,                     # model dimension
    img_encoder = vit,             # vision transformer - image encoder, returning image embeddings as (batch, seq, dim)
    image_dim = 512,               # image embedding dimension, if not the same as model dimensions
    num_tokens = 3,                # number of text tokens
    unimodal_depth = 6,            # depth of the unimodal transformer
    multimodal_depth = 6,          # depth of the multimodal transformer
    dim_head = 64,                 # dimension per attention head
    heads = 8,                     # number of attention heads
    caption_loss_weight = 1.,      # weight on the autoregressive caption loss
    contrastive_loss_weight = 1.,  # weight on the contrastive loss between image and text CLS embeddings
).cuda()

#TODO: need to rescale these basically then we're good
text = torch.from_numpy(y.T).cuda().long()#text = torch.randint(0, 20000, (4, 512)).cuda() .to(device).long() +>(4, 512)
images = torch.from_numpy(X).cuda().long() #images = torch.randn(4, 3, 256, 256).cuda() torch.from_numpy(X) +>(4,3)
i2 = torch.reshape(images, [115, 3, 256, 256])
print(text.size()) #[115, 3]
print(i2.size()) #[115, 256, 256, 3]
#=>pretty sure I just need to move this so it's [115, 3, 256, 256]

#TODO: we fixed cuda problem, now just need to get input shapes right

# train by giving CoCa your text and images with `return_loss = True`

loss = coca(
    text = text,
    images = i2,
    return_loss = True  # set this to True to get the full caption + contrastive loss
)

loss.backward()

# TODO ^ train above on imagenet data

logits = coca(
    text = text,
    images = images
) # (4, 512, 20000)

# and the CLIP-like text and image embeddings as

text_embeds, image_embeds = coca(
    text = text,
    images = images,
    return_embeddings = True
) # (4, 512), (4, 512)

torch.Size([3, 115])
torch.Size([115, 3, 256, 256])


RuntimeError: ignored

PROBLEM ENCOUNTERED: Okay so unless we can find 15gb of RAM we will not be able to train this model to recognise hands, cats and dogs, which is only 13000 images in a 256x256 format. So, let us try using a pre-trained model!

NOTE: Can run code indepentantly from below this text box. Above code is left as prove of work.

In [1]:
#https://drive.google.com/file/d/19z_2hNJkLrbf6AEg7cc6OeazMZq8KEMl/view?usp=sharing
#NOTE: this is all redefined so we don't have to run the above code ^
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#TODO: update with other contender image files
downloaded = drive.CreateFile({'id':'19z_2hNJkLrbf6AEg7cc6OeazMZq8KEMl'}) 
downloaded.GetContentFile('Unstable_Diffusion.zip')

In [2]:
import io
import os 
import zipfile

cd_zip = 'Unstable_Diffusion.zip'
zip_ref = zipfile.ZipFile(cd_zip, 'r')
zip_ref.extractall()
zip_ref.close()

!ls

#according to documentation, expects between [0,255], so don't need to rescale down

sample_data  unstable_diffusion  Unstable_Diffusion.zip


In [3]:
#NOW, need to transform into two arrays, one of all images, one of class labels which are of length 1000
base_dir = 'unstable_diffusion'

In [4]:
#CONSULTING: https://keras.io/api/applications/
import tensorflow as tf
import numpy as np

#returns a keras.model object
model = tf.keras.applications.EfficientNetB7(
    include_top=True,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax"
    #**kwargs
    #^uncomment this to add additional parameters
)

268326632/268326632 [==============================] - 14s 0us/step


In [5]:
#ERROR NOTE
#TODO: we need to change our 3 class representation to map to the imagenet classification array
#if classes become 3 and top = False, size error becomes:Shapes (None, 3) and (None, 18, 18, 2560) are incompatible
#if left default with 1000 and top = True, size error is between (None, 3) and (None, 1000)

#Changing numClasses to 3 (eg. first option) seems correct at face value. How to fix this next error is ???

In [5]:
hardCodeIndexes = [398,413,440,475,661,21,71,113,444]

#TODO: update for more classes
#cringy hard coded formatting stuff but whatever. For now we have 9 classes
def transmuteToImageNetSize(input):
  newArr = [0]*1000

  if(input[0]==1):#abacus 398
    newArr[398] = 1
  elif(input[1]==1):#assault_rifle 413
    newArr[413] = 1
  elif(input[2]==1): #beer_bottle 440
    newArr[440] = 1
  elif(input[3]==1): #car_mirror 475
    newArr[475] = 1
  elif(input[4]==1): #jigsaw_puzzle 661
    newArr[661] = 1
  elif(input[5]==1): #kite 21
    newArr[21] = 1
  elif(input[6]==1): #scorpion 71
    newArr[71] = 1
  elif(input[7]==1): #snail 113
    newArr[113] = 1
  elif(input[8]==1): #tandem_bicycle 444
    newArr[444] = 1
  
  return newArr

In [6]:
#import numpy as np
#import tensorflow as tf
#from tensorflow.keras.preprocessing.image import img_to_array, load_img
#from tensorflow.keras.preprocessing.image import ImageDataGenerator
#import PIL
#from PIL import Image

#TODO: look into alternate methods. Don't need 1/255 rescale
#cdGen = ImageDataGenerator(rescale=1./255)

#recalculated to get 600x600 sizes

#batch size=all our images
#TODO: change preprocessing to be manual. X reshape and no rgb bullshit
#batch_size = len(train_dir) #probably need to do individual ones

#train_generator = cdGen.flow_from_directory(
#    directory=train_dir,
#    target_size=(600, 600),
#    color_mode="rgb",
#    batch_size=batch_size,
#    class_mode="categorical"
#)

#num_samples = train_generator.n
#num_classes = train_generator.num_classes
#input_shape = train_generator.image_shape

#classnames = [k for k,v in train_generator.class_indices.items()]

#X, y = train_generator.next()

#new = []
#for i in y:
#  new.append(transmuteToImageNetSize(i))

#nuY = np.array(new)

In [7]:
#ALTERNATE PREPROCESSING
from tensorflow.keras.utils import load_img
from tensorflow.keras.utils import img_to_array
from tensorflow.keras.utils import array_to_img

#list of directories
abacus_dir = os.path.join(base_dir, 'abacus')
assaultR_dir = os.path.join(base_dir, 'assault_rifle')
beerB_dir = os.path.join(base_dir, 'beer_bottle')
carM_dir = os.path.join(base_dir, 'car_mirror')
jigsawP_dir = os.path.join(base_dir, 'jigsaw_puzzle')
kite_dir = os.path.join(base_dir, 'kite')
scorpion_dir = os.path.join(base_dir, 'scorpion')
snail_dir = os.path.join(base_dir, 'snail')
tandemB_dir = os.path.join(base_dir, 'tandem_bicycle')

In [23]:
#HARDCODE LOAD OF ALL FILES
#NOTE: only doing it this inefficient way because definining it as a list in one step causes a runtime error
#TODO: preprocessing
def load_all_images(dir):
  lst=[]
  for i in os.listdir(dir):
    img = load_img(dir + "/" + i)
    img = img.resize([600,600])
    arr = img_to_array(img)
    lst.append(arr)
  return lst

abacus = load_all_images(abacus_dir)
assault_rifle = load_all_images(assaultR_dir)
beer_bottle = load_all_images(beerB_dir)
car_mirror = load_all_images(carM_dir)
jigsaw_puzzle = load_all_images(jigsawP_dir)
kite = load_all_images(kite_dir)
scorpion = load_all_images(scorpion_dir)
snail =load_all_images(snail_dir)
tandem_bicycle = load_all_images(tandemB_dir)

#these as np arrays all work for classification
abacus_np = np.array(abacus)
assault_rifle_np = np.array(assault_rifle)
beer_bottle_np = np.array(beer_bottle)
car_mirror_np = np.array(car_mirror)
jigsaw_puzzle_np = np.array(jigsaw_puzzle)
kite_np = np.array(kite)
scoprion_np = np.array(scorpion)
snail_np = np.array(snail)
tandem_bicycle_np = np.array(tandem_bicycle) 

In [31]:
import keras
from keras.optimizers import SGD
#TODO: look into different optimizers and how they affect classification and performancev
sgd = SGD(learning_rate=0.01, decay=1e-6, momentum=0.9, nesterov=True)
adam = keras.optimizers.Adam(learning_rate=0.01)

model.compile(loss='categorical_crossentropy',optimizer=adam)

res = model.predict(tandem_bicycle_np)

#TODO: can make a dictionary here
#gets "class" of highest probability using indexes, and probability of such
for i in res:
  class1 = np.argmax(i)
  print("Highest probability class: ", class1)
  print("With probability of: ", i[class1])

#for getting 'top 5' predictions. Can be used if first classification incorrect
#for i in res:
#  print(np.argpartition(i, -5)[-5:])




1/1 [==============================] - 6s 6s/step
Highest probability class: 444
With probability of: 0.71865124
Highest probability class: 444
With probability of: 0.8272607
Highest probability class: 444
With probability of: 0.7445605
Highest probability class: 444
With probability of: 0.72270644
Highest probability class: 444
With probability of: 0.8051849
Highest probability class: 444
With probability of: 0.70570946
Highest probability class: 444
With probability of: 0.7663406


CITATIONS/REFERENCES:

@article{afifi201911kHands, 
title = {11K Hands: gender recognition and biometric identification using a large dataset of hand images}, author = {Afifi, Mahmoud}, journal = {Multimedia Tools and Applications}, doi = {10.1007/s11042-019-7424-8}, url = {https://doi.org/10.1007/s11042-019-7424-8}, year={2019} 

@inproceedings{Yu2022CoCaCC,
  title   = {CoCa: Contrastive Captioners are Image-Text Foundation Models},
  author  = {Jiahui Yu and Zirui Wang and Vijay Vasudevan and Legg Yeung and Mojtaba Seyedhosseini and Yonghui Wu},
  year    = {2022}
}}

@Lots of code referenced from:
https://colab.research.google.com/github/google/eng-edu/blob/master/ml/pc/exercises/image_classification_part1.ipynb#scrollTo=TkNQqjZaB4_u